In [1]:
import pandas as pd

In [3]:
train = pd.read_csv('Data/HorrorAuthor/train.csv')

In [5]:
test = pd.read_csv('Data/HorrorAuthor/test.csv')

In [6]:
train.author.value_counts()

EAP    7900
MWS    6044
HPL    5635
Name: author, dtype: int64

In [127]:
from sklearn.base import BaseEstimator, TransformerMixin
import numpy as np

In [203]:
class WordCountOfText(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass
    
    def text_length(self, name):
        return len(name.split())

    def transform(self, s, y=None):
        return s.apply(self.text_length).values.reshape(-1,1)
    
    def fit(self, s, y=None):
        return self

In [193]:
class ItemSelector(BaseEstimator,TransformerMixin):
    def __init__(self,key):
        self.key = key
        
    def fit(self,x,y=None):
        return self
    
    def transform(self,data_dict):
        return data_dict[self.key]

In [194]:
wc = WordCountOfText()

In [195]:
#wc.fit_transform(train.text)[:5]

In [196]:
from sklearn.pipeline import Pipeline,FeatureUnion
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.naive_bayes import MultinomialNB


In [197]:
enc = Pipeline([
            ('itemselector', ItemSelector('text')),
            ('countVector', CountVectorizer(stop_words='english')),
            ('tfidf', TfidfTransformer())
        ])

In [198]:
wc = Pipeline([
    ('itemselector', ItemSelector('text')),
    ('count', WordCountOfText())
])

In [201]:
pipeline = Pipeline([
    #remove all punchutations
    ('union', FeatureUnion([
            
        ('encoding',enc),
        
        ('wordcount',wc)
        
    ])), #union
    
    ('mnb', MultinomialNB()),
])

In [202]:
pipeline.fit(train,train.author)

Pipeline(memory=None,
     steps=[('union', FeatureUnion(n_jobs=1,
       transformer_list=[('encoding', Pipeline(memory=None,
     steps=[('itemselector', ItemSelector(key='text')), ('countVector', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', inp...    transformer_weights=None)), ('mnb', MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))])

In [204]:
pipeline.predict(test)

array(['MWS', 'EAP', 'EAP', ..., 'EAP', 'MWS', 'EAP'], dtype='<U3')